# Part 1: Data Collection (Ubisoft - External)
- Collecting data that mentions Ubisoft across the various subreddits we are trying to surf from
- Adding prerequisites for the data being collected
    - Post must be more 20 words 
    - Account must be more than 1 week old
    - Account must have more than 10 karma
    - Posts will be collected from the past 1 year


## (1) Import necessary libraries

In [1]:
%pip install praw

Note: you may need to restart the kernel to use updated packages.


In [5]:
import praw
import pandas as pd
import datetime as dt
import csv

import os 
import sys
import csv
from dotenv import load_dotenv


In [3]:
# bring in env variables 
load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
USER_AGENT = os.getenv("USER_AGENT")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

print("Env variables loaded")

Env variables loaded


In [4]:
# initialise connection with reddit
reddit = praw.Reddit(client_id=CLIENT_ID, 
                     client_secret=CLIENT_SECRET, 
                     user_agent=USER_AGENT, 
                     username=USERNAME, 
                     password=PASSWORD)

Version 7.7.1 of praw is outdated. Version 7.8.1 was released 3 days ago.


# (2) Data Scraping 
Within this subsection, we aim to focus mainly on the <strong>EXTERNAL</strong> factors as to why Ubisoft might have failed as a company. We will begin by doing a targeted, unbiased search of all things related to Ubisoft and their games within specific subreddits using neutral keywords. The subreddits that we will be focusing on are 

- r/Ubisoft 
- r/AssassinsCreed
- r/Rainbow6
- r/TheDivision
- r/GhostRecon
- r/ForHonorGame
- r/WatchDogs
- r/Gaming
- r/Games
- r/PCGaming
- r/VideoGameNews
- r/Steam
- r/CrackWatch (for checking how well Ubisoft monitors piracy + bug fixing issues)

The keywords that we will be using to target this search would be 

- (Ubisoft) Ubisoft, Ubi Soft, ubisoft, ubi, ubisofts, Ubisofts, ubis
- (Locations) Ubisoft [geographic location]
- (Platforms) Uplay, UConnect, Ubisoft Connect, Ubisoft Store, Login, PC, Playstation, Xbox, Nintendo Switch, Luna
- (Events) Ubisoft Forward, E3, Showcase, EA + Ubisoft
- (Games/Acronyms) AC, Assassin's Creed, Far Cry, FC5, FC6, Tom Clancy, R6S, R6 Siege, R6 Extraction, Tom Clancy's Rainbow Six, Tom Clancy's The Division, Watch Dogs, Ghost Recon, Just Dance, Prince of Persia, Splinter Cell (this will mainly be used in the r/gaming and more general subreddits), Skull and Bones, Beyond Good and Evil, Beyond Good and Evil 2, Riders Republic, Immortals Fenyx Rising, AC Valhalla, AC Origins, 
- (Game Features) Updates, DLC, Expansion, Patch, season pass, Open World, RPG, multiplayer, speed



In [7]:
# initialise variables 
# 1. subreddits to be scraped 
# 2. neutral keyword to better target searches within posts 

ubisoft_subreddits = ['Ubisoft', 
              "assassinscreed", 
              "Rainbow6", 
              "GhostRecon", 
              "thedivision", 
              "farcry", 
              "farcry5"
              "farcry6", 
              "watch_dogs", 
              "forhonor",
              "Splintercell", 
              "PrinceOfPersia", 
              "JustDance", 
              "Steep", 
              "TrialsGames", 
              "anno", 
              "FenyxRising",
            #   "TheSettlers", 
            #   "beyondgoodandevil", 
              "SkullAndBones",
              "ACValhalla",
              "AssassinsCreedOdyssey",
              "AssassinsCreedOrigins"
              ]

gaming_subreddits = ['gaming',
                     'pcgaming',
                     'PS4',
                     'XboxOne',
                     'NintendoSwitch',
                     'Steam',
                     'PS5',
                     'XboxSeriesX',
                     'CrackWatch']

ubisoft_keywords = ['Ubisoft',
                    ]




20
